# IBKR
# Adding Limit Orders to the Trading Bot (Take Profit & Stop Loss) 

---

# IBKR: Contrader Class without Take Profit & Stop Loss (Recap)

In [1]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

In [3]:
### Start session function 
def start_session():
    global last_update, session_start
    
    last_update = dt.datetime.utcnow() 
    session_start = pd.to_datetime(last_update).tz_localize("utc")
    
    initialize_stream()  
    stop_session()

### Function for initialize streaming data (real time driven data)
def initialize_stream(): 
    global bars, last_bar
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting=freq,
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=2,
            keepUpToDate=True)
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate 
    
### Callback function for catch data streaming updating    
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar, last_update
    
    last_update = dt.datetime.utcnow() 
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass


### Function for execute the trade
def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

### Function for reporting the trade
def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)


### Function for stop the session
def stop_session():
    while True:
        ib.sleep(5) 
        if dt.datetime.utcnow().time() >= end_time:
            execute_trade(target = 0) 
            ib.cancelHistoricalData(bars) 
            ib.sleep(10)
            try:
                trade_reporting() 
            except:
                pass
            print("Session Stopped (planned).")
            ib.disconnect()
            break
        elif dt.datetime.utcnow() - last_update > dt.timedelta(seconds=120):
                # if there was no streaming update in the last 120 seconds
                ib.cancelHistoricalData(bars)
                ib.sleep(5)
                try: # try to reestablish stream
                    initialize_stream()
                except: # stop session
                    ib.sleep(5)
                    try:
                        execute_trade(target = 0) # close open position 
                    except:
                        pass
                    ib.sleep(10)
                    try:
                        trade_reporting() # final reporting
                    except:
                        pass
                    print("Session Stopped - No Connection.")
                    ib.disconnect()
                    break
        else:
            pass

In [4]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [5]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (dt.datetime.utcnow() + dt.timedelta(seconds = 120)).time() # stop condition (5.5 mins from now)
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [6]:
start_session()

,close,returns,position
date,,,
2023-11-07 22:15:00+00:00,1.070045,NaN,NaN
2023-11-07 22:16:00+00:00,1.070020,-0.000023,1.0
2023-11-07 22:17:00+00:00,1.069970,-0.000047,1.0
2023-11-07 22:18:00+00:00,1.070005,0.000033,-1.0
2023-11-07 22:19:00+00:00,1.069975,-0.000028,1.0
...,...,...,...
2023-11-08 06:47:00+00:00,1.068510,0.000019,-1.0
2023-11-08 06:48:00+00:00,1.068630,0.000112,-1.0
2023-11-08 06:49:00+00:00,1.068660,0.000028,-1.0


,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,
2023-11-08 06:52:02+00:00,SLD,1000.0,1.06865,0.0,0.0
2023-11-08 06:53:00+00:00,BOT,1000.0,1.06865,-4.0,-4.0


Session Stopped (planned).


---

# How to create Stop Loss and Take Profit Orders

In [7]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

In [8]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [9]:
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [10]:

def BracketOrder(parentOrderId, childOrderId1, childOrderId2,
                 action, quantity, stopLossPrice = None, takeProfitPrice = None):
    
    """
    A function that prepare order include specified on
    stop loss and take profit. 
    
    Return
    ======
    An order market, include stop loss & take profit orders.  
    - return -> [Order_market, stop_loss_order, take_profit_order]
   
    Parameters
    ==========
    parentOrderId: 
        the market order (short / long)
    childOrderId1: 
        for stop loss order
    childOrderId1: 
        for take profit order    
    action: 
        BUT/SELL 
    quantity:
        Count of the instrument 
    """
    global stopLoss, takeProfit
    
    ## Create market Order (parent) - GO LONG or GO SHORT
    parent = Order()                # create order object
    parent.orderId = parentOrderId  # set market order id
    parent.action = action          # set the action of the order (BUY/SELL)
    parent.orderType = "MKT"        # set order type
    parent.totalQuantity = quantity # set the instrument quantity
    
    if not stopLossPrice and not takeProfitPrice: # if no stop-loss/take-profit specified 
        parent.transmit = True # transmit market order only 
    else:
        parent.transmit = False

    # Store the Market order (parent) in the list.  
    bracketOrder = [parent]

    ## if stop loss specified:
    if stopLossPrice: 
        # attached Stop Loss Order (child) 
        stopLoss = Order()
        stopLoss.orderId = childOrderId1
        stopLoss.action = "SELL" if action == "BUY" else "BUY"
        stopLoss.orderType = "STP"
        stopLoss.auxPrice = stopLossPrice
        stopLoss.totalQuantity = quantity
        stopLoss.parentId = parentOrderId

        if not takeProfitPrice: # if no take profit specified
            stopLoss.transmit = True # transmit mkt order and sl order
        else:
            stopLoss.transmit = False
        # Add the srop loss order to the list 
        bracketOrder.append(stopLoss)
    
    ## if take profit specified:
    if takeProfitPrice:
        # attached Take Profit Order (child)
        takeProfit = Order()
        takeProfit.orderId = childOrderId2
        takeProfit.action = "SELL" if action == "BUY" else "BUY"
        takeProfit.orderType = "LMT"
        takeProfit.totalQuantity = quantity
        takeProfit.lmtPrice = takeProfitPrice
        takeProfit.parentId = parentOrderId
        takeProfit.transmit = True # transmit all three orders
        bracketOrder.append(takeProfit)
        
    return bracketOrder 

In [13]:
# 1. BracketOrder with Mkt Order and SL (optional) and TP (optional)
# Create an order object with spesicing on:
# Order market, take profit, stop loss 

bracket = BracketOrder(parentOrderId = ib.client.getReqId(), # Order market 
                       childOrderId1 = ib.client.getReqId(), # Stop loss
                       childOrderId2 = ib.client.getReqId(), # Take profit
                       action = "BUY",         # Order action (BUY/SELL) 
                       quantity = 1000,        # Quantity of instument
                       stopLossPrice = 1.05,   # Stop loss price limit
                       takeProfitPrice = 1.15, # Take profit price limit
                      )


## This going to return list of orders:
# 1. Market order (inc instument, action, quntity)
# 2. Stop loss (optional)
# 3. take profit (optional)
bracket

[Order(orderId=51, action='BUY', totalQuantity=1000, orderType='MKT', transmit=False),
 Order(orderId=52, action='SELL', totalQuantity=1000, orderType='STP', auxPrice=1.05, transmit=False, parentId=51),
 Order(orderId=53, action='SELL', totalQuantity=1000, orderType='LMT', lmtPrice=1.15, parentId=51)]

In [14]:
## place all orders in BracketOrder
# Its going to send API to the briker and set the orders in live! 
for o in bracket:
    ib.placeOrder(cfd, o)

In [15]:
## close all positions/orders
## We need to close all orders seperdly 

# 1. Cancel the Stop Loss order
try:
    ib.cancelOrder(stopLoss)
except:
    pass

# 2. Cancel the Take Profit order
try:
    ib.cancelOrder(takeProfit)
except:
    pass

# 3.  Cancel Market Order

# If we take long position, we sell the stocks for close the order   
order = MarketOrder("Sell", 1000)
ib.placeOrder(cfd, order)

# If we take short position, we buy the stocks for close the order   
# order = MarketOrder("Buy", 1000)
# ib.placeOrder(cfd, order)

Trade(contract=CFD(conId=143916318, symbol='EUR', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=MarketOrder(orderId=54, clientId=1, action='Sell', totalQuantity=1000), orderStatus=OrderStatus(orderId=54, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2023, 11, 8, 7, 47, 18, 935777, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

---

# IBKR: Stop Loss and Take Profit - Pitfalls and other Considerations
> ### Its just for introduce the idea.

__Be careful__: 
- Stop Loss Price must be below (above) Market Entry Price in Long (Short) position
- Take Profit Price must be above (below) Market Entry Price in Long (Short) position
- Stop Loss and Take Profit Prices must be limited to [5] decimals [EUR/USD]

__Setting Stop Loss Price and Take Profit Price__

Typically: Define Maximum Loss & Take Profit in % (e.g. 1% loss -> 10% if leverage == 10)

In [16]:
# Stop Loss Event if Price moves into the wrong direction by 1%
sl_perc = 0.01 

# Take Profit Event if Price moves into the right direction by 1%
tp_perc = 0.01 

In [17]:
current_price = 1.10

In [22]:
# -1 Short
#  1 Long
#  1 Nutral 
target = 1 # (Target Position: -1/0/1)

In [23]:
if sl_perc:
        if target > 0: # LONG
            sl_price = round(current_price * (1 - sl_perc), 4) 
        elif target < 0: # SHORT
            sl_price = round(current_price * (1 + sl_perc), 4)      
        else: 
            sl_price = None

if tp_perc:
        if target > 0: # LONG
            tp_price = round(current_price * (1 + tp_perc), 4) 
        elif target < 0: # SHORT
            tp_price = round(current_price * (1 - tp_perc), 4)      
        else: 
            tp_price = None

In [24]:
sl_price

1.089

In [25]:
tp_price

1.111

__Identification of SL/TP Events__

Problem in Trading Bot: it doesn´t record TP/SL trades

Solution: Check if expected trading position == actual trading position (if not: very likely SL/TP Event)

---

# IBKR: Adding Stop Loss and Take Profit to the Class

In [2]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

In [3]:
### Function for start the session
def start_session():
    
    # last_update -> the last update from market (date time)
    # session_start -> when the session start (date time)
    # exp_position -> the expected position (LONG/SHORT/NUTRAL)
    # current_pos -> the current position 
    global last_update, session_start, exp_pos, current_pos
    
    # because we start new session, 
    # we need reset all the variables:
    # `expected position`, `current position`, etc..  
    exp_pos = 0     # reset to nutral
    current_pos = 0 # reset to nutral 
    last_update = dt.datetime.utcnow() # reset last update date
    session_start = pd.to_datetime(last_update).tz_localize("utc") # reset session start date
    
    # initialize the streaming data 
    # this is the mechanizem that care to get real time data, and run the strategy.
    initialize_stream()  
    
    # A function for stop the session 
    stop_session()




### Function for initial the streaming data (real time driven data)
def initialize_stream(): 
    """
    A function that care for initial the streaming data (real time driven data), 
    subscribe to an instument, and subscrice the callback function `onBarUpdate()` 
    for catch the streamin data on real time. 
    """

    # bars -> the historical data of an instument.
    # last_bar -> the last bar (the most resent bar that we get from market). 
    global bars, last_bar
    
    # subscribe for get the real driven data of an instument
    bars = ib.reqHistoricalData( 
            contract,               # the instument
            endDateTime='',         #
            durationStr='1 D',      # time span
            barSizeSetting=freq,    # the frequency of bar (1 secs,1 min, etc..)
            whatToShow='MIDPOINT',  #
            useRTH=True,            #
            formatDate=2,           #
            keepUpToDate=True)      # 
    
    # extract the last bar
    last_bar = bars[-1].date 
    
    # subscrice the callback function `onBarUpdate()` to the event (for catch new real time driven data!)
    bars.updateEvent += onBarUpdate 
    

### Calback function that catch events of an instument    
def onBarUpdate(bars, hasNewBar):  
    """
    A function that subscribed to an event (get real time data from the market),
    care to update the data frame, 
    run the strategy, and triger execute order / get order information
    """

    # df -> the data frame that contaon the real time driven data
    # last_bar -> most resent bar date (from market)
    # last_update -> the last update (the time we got new data)
    global df, last_bar, last_update
    
    # update the `last_update` 
    # (if we here, its mean that this function trigered and get a new notification from the event.)
    last_update = dt.datetime.utcnow()
    
    # we check by the dates if we get new bar from the market
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass


### Function for execute the trade
def execute_trade(target): # Modified!!!
    """
    A function for executing a trade. 
    this function send order request to the market for execute the order. 
    this function send specific commends for execute an order that contain: 
        * the market order 
        * the stop loss order (optional)
        * the take profit order (optional)

    Parameters
    ===========
    target:
        the quantity to buy sell. (e.g: 1000 or -1000)

    """

    # the expected position (the position that we want to take) 
    global exp_pos
         
    ## 1. identify required trades
    trades = target - exp_pos
    
    ## 2. determine Stop Loss Price and Take Profit Price
    # current_price -> the most resent close price of the driven data!
    current_price = df.close.iloc[-1]
     

    # Stop loss handling (if we specified stop loss (optonal)) 
    if sl_perc: 
        if target > 0: # LONG
            sl_price = round(current_price * (1 - sl_perc), 4) 
        elif target < 0: # SHORT
            sl_price = round(current_price * (1 + sl_perc), 4)      
    else: 
        sl_price = None

    # Take profit handling (if we specified take profit (optonal)) 
    if tp_perc: # tp_perc -> is take profit precent.
        if target > 0: # LONG
            tp_price = round(current_price * (1 + tp_perc), 4) 
        elif target < 0: # SHORT
            tp_price = round(current_price * (1 - tp_perc), 4)      
    else: 
        tp_price = None


    ## 3. trade execution
    if target > 0: # you want GOING LONG
        if current_pos == 0: # from NEUTRAL
            go_long_short(side = "BUY", target = target, sl_price = sl_price, tp_price = tp_price) # go LONG
        elif current_pos < 0: # from SHORT:
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "BUY", trades = current_pos) # Buy back the stock
            go_long_short(side = "BUY", target = target, sl_price = sl_price, tp_price = tp_price) # then go LONG
  
    elif target < 0: # you want GOING SHORT
        if current_pos == 0: # from NEUTRAL  
            go_long_short(side = "SELL", target = abs(target), sl_price = sl_price, tp_price = tp_price) # go SHORT 
        elif current_pos > 0: # if you came from LONG
            cancel_orders() # cancel sl/tp orders (stop loss/take profit)
            go_neutral(side = "SELL", trades = current_pos) # then sell the stock (from the previes long) 
            go_long_short(side = "SELL", target = abs(target), sl_price = sl_price, tp_price = tp_price) # and then sell new stock
  
    else: # GOING NEUTRAL
        if current_pos < 0: # from SHORT
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "BUY", trades = current_pos)
        elif current_pos > 0: # from LONG:
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "SELL", trades = current_pos)
    exp_pos = target

# Function that prepare & execute LONG/SHORT order (from netural position), inc st/tp 
def go_long_short(side, target, sl_price, tp_price): # NEW Go Long/Short starting from Neutral posistion
    """
    A function that create and execute an order with a given parameters. 

    Parameters
    ==========
    side:
      the order action (LONG/SHORT)  
    target:
      the quantity of an instument (e.g: 1000 or -1000)
    sl_price:
      the price that we want to locate the stop loss
    tp_price:
      the price that we want to locate the take profit

    """
    # we call the function `BracketOrder()` for create list of orders [Market order, st order (optional), tp order (optional)] 
    bracket = BracketOrder(parentOrderId = ib.client.getReqId(), 
                           childOrderId1 = ib.client.getReqId(), 
                            childOrderId2 = ib.client.getReqId(),
                            action = side,
                            quantity = target,
                            stopLossPrice = sl_price, 
                            takeProfitPrice = tp_price,
                          )
    # then, we execute each order [Market order, st order (optional), tp order (optional)] 
    for o in bracket:
        # cfd -> the instument.
        # o -> the order.
        order = ib.placeOrder(cfd, o)


### Function for close LONG/SHORT positions (go neutal)
def go_neutral(side, trades): # Close Long/Short position
    """
    A function that close LONG/SHORT position

    Parameters
    ==========
    side:
      LONG/SHORT
    trades:
      quantity of the instument
    """
    
    # Create the order
    order = MarketOrder(side, abs(trades))
    
    # execute the order
    trade = ib.placeOrder(cfd, order)    
    

### Function for cancel orders (SL/TP orders)
def cancel_orders(): # cancel SL/TP orders
    try:
        # cancel the stop loss order
        sl_cancel = ib.cancelOrder(stopLoss) 
    except:
        pass
    try:
        # cancel the take profit order
        tp_cancel = ib.cancelOrder(takeProfit)
    except:
        pass 

### Function for create orders 
def BracketOrder(parentOrderId, childOrderId1, childOrderId2,
                 action, quantity, stopLossPrice, takeProfitPrice): # NEW
    
    """
    Function for create orders: [Market order, sl order (optional), tp order (optional)].

    Parameters
    ==========
    parentOrderId: 
        Market order
    childOrderId1:
        Stop loss order (optional)
    childOrderId2:
        Take profit order (optional)  
    action: 
        LONG/SHORT  
    quantity: 
        Count of the instument
    stopLossPrice: 
        The price limit for stop loss
    takeProfitPrice:
        The price limit for take profit
    
    """

    # stopLoss -> the stop loss order
    # takeProfit -> the take profit order
    global stopLoss, takeProfit
    
    # Market Order (parent) - GO LONG or GO SHORT
    parent = Order()
    parent.orderId = parentOrderId
    parent.action = action
    parent.orderType = "MKT"
    parent.totalQuantity = quantity
    if not stopLossPrice and not takeProfitPrice: 
        parent.transmit = True
    else:
        parent.transmit = False
        
    bracketOrder = [parent]

    if stopLossPrice:
        # attached Stop Loss Order (child) 
        stopLoss = Order()
        stopLoss.orderId = childOrderId1
        stopLoss.action = "SELL" if action == "BUY" else "BUY"
        stopLoss.orderType = "STP"
        stopLoss.auxPrice = stopLossPrice
        stopLoss.totalQuantity = quantity
        stopLoss.parentId = parentOrderId
        if not takeProfitPrice: 
            stopLoss.transmit = True
        else:
            stopLoss.transmit = False
        bracketOrder.append(stopLoss)
    
    if takeProfitPrice:
        # attached Take Profit Order (child)
        takeProfit = Order()
        takeProfit.orderId = childOrderId2
        takeProfit.action = "SELL" if action == "BUY" else "BUY"
        takeProfit.orderType = "LMT"
        takeProfit.totalQuantity = quantity
        takeProfit.lmtPrice = takeProfitPrice
        takeProfit.parentId = parentOrderId
        takeProfit.transmit = True
        bracketOrder.append(takeProfit)
        
    return bracketOrder 
    
### Function for report on the trade     
def trade_reporting():
    global report
    
    # Create data frame that contain the report data
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "shares", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby(["time", "side"]).agg({"shares":"sum", "avgPrice":"mean", "realizedPNL":"sum"}).reset_index().set_index("time")
    report["cumPNL"] = report.realizedPNL.cumsum()

    # Clear & display 
    clear_output(wait=True)
    display(df, report)


### Function for stop session  
def stop_session():
    """
    Function that stop the session, buy chack the state each 5 minutes. 
    """

    # the current position 
    global current_pos
    

    while True:
        # loop each 5 minutes
        ib.sleep(5) 
        
        # try to get the current position (from the market)
        try:
            current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
        except:
            current_pos = 0
       
        # Check if the session over the time limit that we specified
        if dt.datetime.utcnow().time() >= end_time:
            # target = 0 => cancel any order (go neutral)
            execute_trade(target = 0) 
            ib.cancelHistoricalData(bars) 
            ib.sleep(10)
            
            # get a report
            try:
                trade_reporting() 
            except:
                pass

            # disconnect
            print("Session Stopped (planned).")
            ib.disconnect() 
            break

        
        ### if SL/TP Event
        #### If we run into Stop Loss / Take Profit event -> we shuld stop the session !!! ### 
        # What we want to check here?
        # - whether the stop loss/take proffit are triggered!!!
        # so we get the current position from IBKR, 
        # and then we compare the expected position as recorded by our trading bot. 
        elif exp_pos != current_pos: 
            ib.sleep(5)
            try:
                # Get current positions
                current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
            except:
                current_pos = 0
              
            # if its still the situation, cancell the orders  
            if exp_pos != current_pos:
                execute_trade(target = 0) 
                ib.cancelHistoricalData(bars) 
                ib.sleep(10)
                try:
                    trade_reporting() 
                except:
                    pass
                print("Session Stopped (SL/TP Event).")
                ib.disconnect()
                break
            else:
                pass
         
        # If the session time above the limit, stop the trading session
        elif dt.datetime.utcnow() - last_update > dt.timedelta(seconds=120):
                # if there was no streaming update in the last 120 seconds
                ib.cancelHistoricalData(bars)
                ib.sleep(5)
                try: # try to reestablish stream
                    initialize_stream()
                except: # stop session
                    ib.sleep(5)
                    try:
                        execute_trade(target = 0) # close open position 
                    except:
                        pass
                    ib.sleep(10)
                    try:
                        trade_reporting() # final reporting
                    except:
                        pass
                    print("Session Stopped - No Connection.")
                    ib.disconnect()
                    break
        else:
            pass

In [4]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [5]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (dt.datetime.utcnow() + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
sl_perc = 0.001
tp_perc = 0.001
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [6]:
start_session()

,close,returns,position
date,,,
2023-11-07 22:15:00+00:00,1.070045,NaN,NaN
2023-11-07 22:16:00+00:00,1.070020,-0.000023,1.0
2023-11-07 22:17:00+00:00,1.069970,-0.000047,1.0
2023-11-07 22:18:00+00:00,1.070005,0.000033,-1.0
2023-11-07 22:19:00+00:00,1.069975,-0.000028,1.0
...,...,...,...
2023-11-08 11:51:00+00:00,1.067310,-0.000005,1.0
2023-11-08 11:52:00+00:00,1.067415,0.000098,-1.0
2023-11-08 11:53:00+00:00,1.067565,0.000141,-1.0


,side,shares,avgPrice,realizedPNL,cumPNL
time,,,,,
2023-11-08 11:51:01+00:00,SLD,1000.0,1.06730,0.00,0.00
2023-11-08 11:52:01+00:00,BOT,2000.0,1.06735,-4.05,-4.05
2023-11-08 11:53:06+00:00,SLD,2000.0,1.06740,-3.95,-8.00
2023-11-08 11:56:29+00:00,BOT,1000.0,1.06775,-4.35,-12.35


Session Stopped (planned).
